In [836]:
def clearing(data):
    import pandas as pd
    import re
    import ast
    top = ['WA', 'NH', 'MN', 'UT', 'VT', 'MD', 'VA', 'MA', 'NE', 'CO', 'WI', 'NJ', 'FL', 'IA', 'ND', 'ID']
    mid = ['GA', 'NC', 'CA', 'SD', 'CT', 'KS', 'DE', 'HI', 'NY', 'RI', 'OR', 'MO', 'MT', 'TN', 'WY', 'ME']
    data = data.drop(['MlsId', 'mls-id', 'private pool', 'PrivatePool', 'propertyType', 'status'],axis = 1)
    def state(x):
        if x in top:
            return 2
        if x in mid:
            return 1
        return 0
    
    def func_cool(x):
        if x == '':
            return 0
        return 1

    def func_pool(x):
        if type(x) != int:
            x = x.replace(x, '1')
        return int(x)
    
    def func_stor(x):
        if type(x) != int:
            x = re.findall(r'\d+',x)
            if (len(x) > 1):
                x = x[0]
            elif len(x) == 1:
                x = x[0]
            if not x:
                return 1
        return x
    
    def func_area(x):
        if type(x) != int:
            x = re.findall(r'\d+',x)
            if len(x) > 1:
                x = x[0] + x[1]
            elif len(x) == 1:
                x = x[0]
            if not x:
                return 4940
        return int(x)

    def func_lot(x):
        if type(x) != int:
            x = re.findall(r'\d+',x)
            if len(x) > 1:
                x = x[0] + x[1]
            elif len(x) == 1:
                x = x[0]
            if not x:
                return 5200
        return int(x)
    
    def func_built(x):
        x = re.findall(r'\d+',x)
        if len(x) > 1:
            x = x[0] + x[1]
        elif len(x) == 1:
            x = x[0]
        if not x:
            return 1976
        return int(x)
        
    def data_func1(x):
        a = ast.literal_eval(x)
        return a['atAGlanceFacts'][0].get('factValue')

    def data_func2(x):
        a = ast.literal_eval(x)
        return a['atAGlanceFacts'][1].get('factValue')

    def data_func6(x):
        a = ast.literal_eval(x)
        return a['atAGlanceFacts'][5].get('factValue')
    
    def data_func8(x):
        m = []
        a = ast.literal_eval(x)
        a = dict(a[0])
        k = a['data'].get('Distance')
        for i in k:
            h = re.findall(r'\d+',i)
            if len(h) == 0:
                continue
            if len(h) == 1:
                m.append(float(h[0]))
                continue
            d = h[0] + '.' + h[1]
            m.append(float(d))
        if len(m) == 0:
            return 1.1
        else:
            return min(m)
    
    def func_rebuilt(row):
        if row['rebuilt'] == 0:
            return row['built']
        if row['rebuilt'] == '':
            return row['built']
        return row['rebuilt']
 
    data['fireplace'] = data['fireplace'].fillna(0).apply(func_pool)
    data['baths'] = data['baths'].fillna(1).apply(func_stor)
    data['stories'] = data['stories'].fillna(1).apply(func_stor)
    data['beds'] = data['beds'].fillna(1).apply(func_stor)
    data['built'] = data['homeFacts'].apply(data_func1)
    data['rebuilt'] = data['homeFacts'].apply(data_func2)
    data['lotsize'] = data['homeFacts'].apply(data_func6)
    data['distance'] = data['schools'].apply(data_func8)
    data = data.drop(['homeFacts', 'schools'], axis = 1)
    data['state'] = data['state'].apply(state)
    data['lotsize'] = data['lotsize'].fillna(5200).apply(func_lot)
    data['built'] = data['built'].fillna(1976)
    data['built'] = data['built'].astype(str)
    data['built'] = data['built'].apply(func_built)
    data['built'] = data['built'].astype(int)
    data['sqft'] = data['sqft'].fillna(4940).apply(func_area)
    data['rebuilt'] = data['rebuilt'].fillna(0)
    data['rebuilt'] = data.apply(lambda row: func_rebuilt(row), axis=1)
    data['rebuilt'] = data['rebuilt'].astype(int)
    data['baths'] = data['baths'].astype(int)
    data['beds'] = data['beds'].astype(int)
    data['stories'] = data['stories'].astype(int)
    return data

In [837]:
import pandas as pd
import pickle
import numpy as np
filename = 'data.csv'
data = pd.read_csv(filename)
data = clearing(data)
totrain = pd.read_csv('obr.csv')
totrain = totrain.drop(['status', 'propertyType', 'heating', 'cooling', 'parking', 'PrivatePool', 'target'], axis = 1)
data = data.drop(data.loc[data['city'].isnull()].index)
data = data.drop(data.loc[data['street'].isnull()].index)
data = data.drop(data.loc[data['zipcode'].isnull()].index)
full = pd.concat([data, totrain], ignore_index=True)
numerics = ['int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorical_columns = []
features = full.columns.values.tolist()
for col in features:
    if full[col].dtype in numerics: continue
    categorical_columns.append(col)
for col in categorical_columns:
    if col in full.columns:
        le = LabelEncoder()
        le.fit(list(full[col].astype(str).values))
        full[col] = le.transform(list(full[col].astype(str).values))
with open('mymodel.pkl', 'rb') as pkl_file:
    regressor = pickle.load(pkl_file)
X = full[:len(data)]
pred = regressor.predict(X)
for x in range(len(pred)):
    pred[x] = round(pred[x]/1000)*1000
target = pd.DataFrame(pred)
final = X.join(target)
final.head()